# Determine weekly nakeds
* A shorter version of nakeds

In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_ohlcs.pkl', 'df_opt_margins.pkl',
       'df_opt_prices.pkl', 'df_opts.pkl', 'df_symlots.pkl',
       'df_und_margins.pkl', 'df_unds.pkl', 'qopt_rejects.pkl',
       'qopts.pkl'], dtype='<U18')

In [7]:
# * IMPORTS
from support import Timer, calcsdmult_df, get_openorders
from engine import get_unds
from ib_insync import LimitOrder

In [8]:
# * FUNCTION INPUTS

In [9]:
# * SETUP
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# ...setup logs and clear them
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_weeklies.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

# ... load files
df_opts = pd.read_pickle(DATAPATH.joinpath('df_opts.pkl'))
df_symlots = pd.read_pickle(DATAPATH.joinpath('df_symlots.pkl'))

In [10]:
from engine import get_symlots
get_symlots(MARKET)


snp symlot qualification started at 27-Nov-2020 22:30:49


qualify:  100%|██████████████████████████████| 254/254 [00:06<00:00, 40.79it/s]                                        


...snp symlot qualification took: 00:00:08 seconds



,symbol,secType,exchange,expiry,lot,currency,contract
0,UPRO,STK,SMART,None,100,USD,"Contract(secType='STK', conId=61228752, symbol..."
1,TNA,STK,SMART,None,100,USD,"Contract(secType='STK', conId=55679435, symbol..."
2,XME,STK,SMART,None,100,USD,"Contract(secType='STK', conId=45540699, symbol..."
3,XLF,STK,SMART,None,100,USD,"Contract(secType='STK', conId=4215220, symbol=..."
4,YINN,STK,SMART,None,100,USD,"Contract(secType='STK', conId=70737252, symbol..."
...,...,...,...,...,...,...,...
248,WYNN,STK,SMART,None,100,USD,"Contract(secType='STK', conId=16454492, symbol..."
249,XLNX,STK,SMART,None,100,USD,"Contract(secType='STK', conId=276222, symbol='..."
250,XOM,STK,SMART,None,100,USD,"Contract(secType='STK', conId=13977, symbol='X..."
251,YUM,STK,SMART,None,100,USD,"Contract(secType='STK', conId=3206042, symbol=..."


In [ ]:
# * RECONSTRUCT sdmult AND rom

# ... remove YAML blacklisted symbols
df_opts = df_opts[~df_opts.symbol.isin(ibp.BLACKLIST)]

df_unds = get_unds(MARKET=MARKET, und_cts=df_symlots.contract.unique(), savedf = True, RUN_ON_PAPER = False)

# ... pad ivs
no_ivs = df_opts[df_opts.iv.isnull()].symbol.unique()
iv_dict = df_unds[df_unds.symbol.isin(no_ivs)].set_index('symbol').iv.to_dict()
df_opts = df_opts.assign(iv=df_opts.symbol.map(iv_dict).fillna(df_opts.iv))

# ... recompute sdMults
df_opts = df_opts.assign(sdMult=calcsdmult_df(df_opts.strike, df_opts))

# ... pad option margins with und_margins where it is not available
no_margins = df_opts[df_opts.margin.isnull()].symbol.unique()
mgn_dict = df_unds[df_unds.symbol.isin(no_margins)].set_index('symbol').margin.to_dict()
df_opts = df_opts.assign(margin=df_opts.symbol.map(mgn_dict).fillna(df_opts.margin))

# ... update df_opts with the latest undPrice
und_price_dict = df_unds.set_index('symbol').undPrice.to_dict()
df_opts = df_opts.assign(undPrice=df_opts.symbol.map(und_price_dict).fillna(df_opts.undPrice))

# ... recalc intrinsic and time values
df_opts = df_opts.assign(intrinsic=np.where(
                df_opts.right == "C",
                (df_opts.undPrice - df_opts.strike).clip(0, None),
                (df_opts.strike - df_opts.undPrice).clip(0, None),
                ))

df_opts = df_opts.assign(timevalue=df_opts.price - df_opts.intrinsic)

# ... recompute roms
df_opts = df_opts.assign(rom = (df_opts.timevalue * df_opts.lot - df_opts.comm).clip(0) /
            df_opts.margin * 365 / df_opts.dte)
df_opts = df_opts.sort_values('rom', ascending=False).reset_index(drop=True)


# * GET TARGETS
# ... expected rom and sdMult mask
m = (df_opts.rom > ibp.MINEXPROM) & np.where(df_opts.right == 'P', df_opts.sdMult > ibp.PUTSTDMULT, df_opts.sdMult > ibp.CALLSTDMULT)

# ... right mask
r = (df_opts.strike > df_opts.undPrice) & (df_opts.right == 'C') | (df_opts.strike < df_opts.undPrice) & (df_opts.right == 'P')
df_opts1 = df_opts[m & r].sort_values(['right', 'strike'], ascending=[True, False])

#.. add expPrice
df_opts1['expPrice'] = df_opts1.price.clip(ibp.MINOPTSELLPRICE)

In [ ]:
# SHOW UNIQUE SYMBOLS
np.array(df_opts1.symbol.unique())

In [ ]:
# CHECK SPECIFIC expPrice
df_opts1.drop('contract', 1)[df_opts1.symbol == 'XSP'].sort_values('rom', ascending=False)

In [ ]:
# CHANGE SPECIFIC expPrice
df_opts1.loc[df_opts1.symbol == 'GPS', 'expPrice'] = 0.95

In [ ]:
# ... check open ordes and remove them
df_openords = get_openorders(MARKET)
df1 = df_opts1[~df_opts1.conId.isin(df_openords.conId)].reset_index(drop=True)

In [ ]:
# ... build the SELL orders
contracts = df1.contract.to_list()
orders = [LimitOrder(action='SELL', totalQuantity=abs(int(q)), lmtPrice=p) 
                        for q, p in zip(df1.qty, df1.expPrice)]

cos = [(c, o) for c, o in zip(contracts, orders)]

# Place the trades